<a href="https://colab.research.google.com/github/Charmie13/MSCI-641-Text-Analytics_Final-Project/blob/main/XLNet_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install simpletransformers

In [ ]:
import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

In [ ]:
train_set = pd.read_csv('train_reduced.csv')
test_set = pd.read_csv('test.csv')

In [ ]:
train_set

In [ ]:
train = pd.DataFrame(columns=['text_a', 'text_b', 'labels'])
train['text_a'] = train_set['Headline']
train['text_b'] = train_set['articleBody']

In [ ]:
stances = train_set['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances
train['labels'] = stances

In [ ]:
train

In [ ]:
stances = test_set['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances

In [ ]:
test = pd.DataFrame(columns=['text_a', 'text_b'])

test['text_a'] = test_set['Headline']
test['text_b'] = test_set['articleBody']

In [ ]:
train

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=4, args={
    'learning_rate':2e-5,
    'num_train_epochs': 5,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'evaluate_during_training_steps': False,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'save_best_model': True,
    'warmup_ratio':0.1,
    'weight_decay':0.01,
    'load_best_model_at_end':True,
    'fp16': False,
    'use_cuda': False,
    'early_stopping' : True
})

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
model.train_model(train)

  0%|          | 0/26854 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

(33570, 0.18082699972035648)

In [ ]:
!pwd

/content


In [ ]:
test_11 = list()
for i in range(len(test)):
  test_11.append([test['text_a'][i], test['text_b'][i]])

In [ ]:
predictions, raw_outputs = model.predict(test_11)

  0%|          | 0/25413 [00:00<?, ?it/s]

  0%|          | 0/6354 [00:00<?, ?it/s]

In [ ]:
cnt = 0
for i in range(len(predictions)):
  if predictions[i] == stances[i]:
    cnt = cnt + 1
print(cnt/len(predictions))

0.9222051705819856


In [ ]:
answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
answer['Stance'] = predictions
answer.replace(0, 'agree',True)
answer.replace(1, 'disagree',True)
answer.replace(2, 'discuss',True)
answer.replace(3, 'unrelated',True)
answer['Headline'] = test_set['Headline']
answer['Body ID'] = test_set['Body ID']

In [ ]:
test_update = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
test_update['Stance'] = test_set['Stance']
test_update.replace(0, 'agree',True)
test_update.replace(1, 'disagree',True)
test_update.replace(2, 'discuss',True)
test_update.replace(3, 'unrelated',True)
test_update['Headline'] = test_set['Headline']
test_update['Body ID'] = test_set['Body ID']

In [ ]:
test_update

In [ ]:
answer.to_csv('answer.csv', index=False, encoding='utf-8')

In [ ]:
!git clone https://github.com/FakeNewsChallenge/fnc-1-baseline.git

Cloning into 'fnc-1-baseline'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 23.99 KiB | 11.99 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
cd fnc-1-baseline/

/content/fnc-1-baseline


In [ ]:
from utils.score import report_score,score_submission

In [ ]:
actual = test_update['Stance'].values
actual

array(['unrelated', 'unrelated', 'unrelated', ..., 'disagree', 'disagree',
       'agree'], dtype=object)

In [ ]:
df_pred = pd.read_csv('../answer.csv')
pred = df_pred['Stance'].values
print("F1 Score for XLNet:")
report_score(actual, pred)

F1 Score for XLNet:
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |   1380    |    80     |    413    |    30     |
-------------------------------------------------------------
| disagree  |    181    |    332    |    144    |    40     |
-------------------------------------------------------------
|  discuss  |    487    |    191    |   3693    |    93     |
-------------------------------------------------------------
| unrelated |    53     |    21     |    244    |   18031   |
-------------------------------------------------------------
Score: 10286.75 out of 11651.25	(88.2888102134964%)


88.2888102134964